In [100]:
import numpy as np
import matplotlib.pyplot as plt
import astropy 
from astropy.io import fits
from astropy.table import Table, Column, vstack, MaskedColumn, QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
from astropy.io import ascii
import os
import seaborn as sns
print(os.environ['PATH'])
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
print(os.environ['PATH'])
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 11
color_palette = sns.color_palette(["#000000", "#849324", "#FFB30F", "#D62828", "#6EA2D2"])

/Library/TeX/texbin:/Users/cassiemetzger/opt/anaconda3/bin:/Library/TeX/texbin:/Users/cassiemetzger/.gem/ruby/3.1.3/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/lib/ruby/gems/3.1.0/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/bin:/Users/cassiemetzger/opt/anaconda3/bin:/Users/cassiemetzger/opt/anaconda3/condabin:/opt/local/bin:/opt/local/sbin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin:/Library/Apple/usr/bin
/Library/TeX/texbin:/Library/TeX/texbin:/Users/cassiemetzger/opt/anaconda3/bin:/Library/TeX/texbin:/Users/cassiemetzger/.gem/ruby/3.1.3/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/lib/ruby/gems/3.1.0/bin:/Users/cassiemetzger/.rubies/ruby-3.1.3/bin:/Users/cassiemetzger/opt/anaconda3/bin:/Users/cassiemetzger/opt/anaconda3/condabin:/opt/local/bin:/opt/local/sbin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/tex

In [108]:
hdu_list = fits.open('./catalogs/IR/wise.fits')
wise = Table(hdu_list[1].data)
hdu_list.close()

In [102]:
hdu_list = fits.open('./catalogs/xray/eRASS1_Main.v1.1.fits')
erass1 = Table(hdu_list[1].data)
hdu_list.close()

In [112]:
c = SkyCoord(ra = wise['ra']*u.degree, dec = wise['dec']*u.degree, frame = 'icrs')
catalog = SkyCoord(ra = erass1['RA']*u.degree, dec = erass1['DEC']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <= 5 
wise['erass idx'] = idx 
matches = wise[m]

In [209]:
#assumes RA, DEC coords are given in degrees and radius is given in arcseconds
def matching(catalog1, RA1, DEC1, catalog2, RA2, DEC2, radius, matching_Name): 
    c = SkyCoord(ra = RA1*u.degree, dec = DEC1*u.degree, frame = 'fk5')
    catalog = SkyCoord(ra = RA2*u.degree, dec = DEC2*u.degree, frame = 'fk5')
    idx, d2d, d3d = c.match_to_catalog_sky(catalog)
    mask = d2d.arcsec <= radius
    matches = catalog1[mask]
    matches[f'{matching_Name} idx'] = idx[mask]
    matches[f'{matching_Name} sep'] = d2d.arcsec[mask]
    leng = len(matches)
    print(f'{leng} {matching_Name} matches found')
    print('Checking for duplicates...')
    numbers= matches[f'{matching_Name} idx']
    unique, counts = np.unique(numbers, return_counts=True)
    duplicates = unique[counts > 1] 
    print(f"Duplicates found: {len(duplicates)}")
    if(len(duplicates > 0)): 
        print(f"Removing duplicates...")
        mask = np.array([False if i in duplicates else True for i in matches[f'{matching_Name} idx']])
        for val in duplicates: 
            idx = np.where(matches[f'{matching_Name} idx'] == val)[0]
            dis1 = matches[f'{matching_Name} sep'][idx[0]]
            dis2 = matches[f'{matching_Name} sep'][idx[1]]
            if(dis1 < dis2): 
                mask[idx[0]] = True
                mask[idx[1]] = False
            if(dis2 < dis1): 
                mask[idx[0]] = False
                mask[idx[1]] = True
            if(dis1 == dis2): 
                mask[idx[0]] = False
                mask[idx[1]] = True
        matches = matches[mask]
        print(f"Removed {leng- len(matches)} duplicates \n {len(matches)} {matching_Name} matches found")
    return matches

In [215]:
matches = matching(wise, wise['ra'], wise['dec'], erass1, erass1['RA'], erass1['DEC'], 5, 'WISE')

5634 WISE matches found
Checking for duplicates...
Duplicates found: 152
Removing duplicates...
Removed 152 duplicates 
 5482 WISE matches found


In [216]:
def get_wise_flux(catalog): 
    sol = 2.997925E14 # variable sol; c in microns/s
    w1=sol/3.368
    w2=sol/4.618
    w3=sol/12.082

    fc_w1=0.9921
    fc_w2=0.9943
    fc_w3=0.9373

    F_w1 = 306.682 #Jy
    F_w2 = 170.663 #Jy
    F_w3 = 29.045 #Jy

    flux_w1 = F_w1/fc_w1*10**(-catalog['w1mpro']/2.5)
    flux_w2 = F_w2/fc_w2*10**(-catalog['w2mpro']/2.5)
    flux_w3 = F_w3/fc_w3*10**(-catalog['w3mpro']/2.5)

    #errors 
    w1errh = catalog['w1mpro'] - catalog['w1sigmpro']
    w2errh = catalog['w2mpro'] - catalog['w2sigmpro']
    w3errh = catalog['w3mpro'] - catalog['w3sigmpro']

    w1errl = catalog['w1mpro'] + catalog['w1sigmpro']
    w2errl = catalog['w2mpro'] + catalog['w2sigmpro']
    w3errl = catalog['w3mpro'] + catalog['w3sigmpro']

    #relative errors in Jy 

    err_w1_h = F_w1/fc_w1*10**(-w1errh/2.5)-flux_w1
    err_w2_h = F_w2/fc_w2*10**(-w2errh/2.5)-flux_w2
    err_w3_h = F_w3/fc_w3*10**(-w3errh/2.5)-flux_w3

    err_w1_l = F_w1/fc_w1*10**(-w1errl/2.5)-flux_w1
    err_w2_l = F_w2/fc_w2*10**(-w2errl/2.5)-flux_w2
    err_w3_l = F_w3/fc_w3*10**(-w3errl/2.5)-flux_w3

    err_w1 = 0.5*(np.abs(err_w1_h)+ np.abs(err_w1_l))
    err_w2 = 0.5*(np.abs(err_w2_h)+np.abs(err_w2_l))
    err_w3 = 0.5*(np.abs(err_w3_h)+np.abs(err_w3_l))

    flux = [flux_w1,flux_w2,flux_w3]
    err = [err_w1, err_w2, err_w3]
    catalog['w1flux'] = flux_w1 
    catalog['w2flux'] = flux_w2 
    catalog['w3flux'] = flux_w3
    catalog['w1sigflux'] = err_w1
    catalog['w2sigflux'] = err_w2
    catalog['w3sigflux'] = err_w3
    return catalog


In [217]:
matches = get_wise_flux(matches)

In [218]:
matches['ML_FLUX_1'] = erass1['ML_FLUX_1'][matches['erass idx']]

In [219]:
#choosing erosita 0.2-2.3 keV energy band, Eband = 1 keV 
e_energy = 1.6022e-16 #J 
h = 6.626e-34 #Js
e_v1 = e_energy/h
w_v = 3E8/3.4E-6


In [220]:
matches['ML_FLUX_1_JY'] = 1E+23 * matches['ML_FLUX_1'] * 1/(e_v1)
matches['Alpha IRX'] = -np.log10(matches['w1flux']/matches['ML_FLUX_1_JY'])/np.log10(w_v/e_v1)

In [221]:
matches['EXT_LIKE'] = erass1['EXT_LIKE'][matches['erass idx']]

In [222]:
x = matches['w2mpro'] - matches['w3mpro']
y = matches['w1mpro'] - matches['w2mpro']
m = 0.83809562126163
b = -1.045969175938493
def distance (m,b, x,y): 
   return np.abs(m * x + b-y)/np.sqrt(m**2 + 1**2)
matches['dist'] = distance(m,b, x,y)

In [223]:
matches['w2 - w3'] = x 
matches['w1 - w2'] = y

In [224]:
hdu_list = fits.open('./catalogs/THC_catalog.fits')
thc = Table(hdu_list[1].data)
hdu_list.close()

In [225]:
matches['THC'] = Column([''] * len(matches), dtype = 'U9')
for i in range(len(matches)): 
    if(matches['name'][i] in thc['WISEA']): 
        idx = np.where(thc['WISEA'] == matches['name'][i])[0][0]
        matches['THC'][i] = thc['THC'][idx]

In [226]:
matches['1eRASS'] = erass1['IAUNAME'][matches['erass idx']]

In [227]:
name = matches['1eRASS'][0]
name[7:23]

'J051642.1-844007'

In [228]:
matches['1eRASS'] = [name[7:23] for name in matches['1eRASS']]

In [229]:
tevcat= ascii.read('./catalogs/other_blazar-hbl_cats/HBL_newlist.csv', format='csv')

In [230]:
c = SkyCoord(ra = matches['ra']*u.degree, dec = matches['dec']*u.degree, frame = 'icrs')
catalog = SkyCoord(ra = tevcat['Simbad RA deg']*u.degree, dec = tevcat['Simbad DEC deg']*u.degree, frame = 'icrs')
idx, d2d, d3d = match_coordinates_sky(c, catalog)
m = d2d.arcsec <= 5
matches['TeVCAT Assoc'] = Column([''] * len(matches), dtype = 'U21')
matches['TeVCAT Assoc'][m] = tevcat['Name'][idx[m]]


In [231]:
final = Table([matches['THC'], matches['name'], matches['1eRASS'], matches['ra'], matches['dec'], matches['EXT_LIKE'], matches['w2 - w3'], matches['w1 - w2'], matches['dist'], matches['Alpha IRX'], matches['TeVCAT Assoc']])

In [232]:
final

THC,name,1eRASS,ra,dec,EXT_LIKE,w2 - w3,w1 - w2,dist,Alpha IRX,TeVCAT Assoc
str9,str19,str16,float64,float64,float32,float64,float64,float64,float64,str21
,J051639.13-844008.0,J051642.1-844007,79.1630516,-84.6688941,0.0,2.349,0.3439999999999994,0.44354247861439267,1.1527534338858887,
,J185159.49-841628.2,J185158.8-841625,282.9978981,-84.2745041,0.0,2.302999999999999,0.5470000000000006,0.25841111316630005,1.248067313632769,
,J075655.94-815653.4,J075655.2-815654,119.2330998,-81.948188,0.0,1.4100000000000001,0.20599999999999952,0.05384456462967376,1.4068596786153855,
,J091009.53-794627.6,J091008.7-794628,137.539722,-79.7743393,0.0,1.6300000000000008,0.14700000000000024,0.13268831150623522,1.9158074073117017,
,J043124.15-793224.6,J043122.6-793225,67.8506267,-79.5401767,0.0,2.3889999999999993,0.7829999999999995,0.13277612562430968,1.2668721356997477,
,J130722.72-773722.9,J130723.2-773724,196.8446743,-77.6230362,0.0,1.6749999999999998,0.5739999999999998,0.16566928323212626,2.034770985471037,
,J082029.61-800325.6,J082029.0-800327,125.1233979,-80.0571241,0.0,1.8989999999999991,0.3930000000000007,0.1169365686566224,1.7524235326117097,
,J011947.05-790356.1,J011946.5-790354,19.9460557,-79.0655836,0.0,2.2990000000000013,0.7479999999999993,0.10179070357924803,1.212621320694974,
,J133051.33-770928.2,J133051.3-770930,202.7138944,-77.1578497,0.0,1.9540000000000006,0.20400000000000063,0.2971190325717619,1.3685424826353647,


In [233]:
hdu_list = fits.open('./catalogs/gammaray/2FHL.fits')
fhl2 = Table(hdu_list[1].data)
c = SkyCoord(ra = final['ra']*u.degree, dec = final['dec']*u.degree, frame = 'fk5')
catalog = SkyCoord(ra = fhl2['RAJ2000']*u.degree, dec = fhl2['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl2['Pos_err_95'][idx]
final['2FHL Assoc'] = Column(['']*len(final), dtype = 'U17')
final['2FHL Assoc'][mask] = fhl2['Source_Name'][idx[mask]]
hdu_list = fits.open('./catalogs/gammaray/3FHL.fits')
fhl3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fhl3['RAJ2000']*u.degree, dec = fhl3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl3['Conf_95_SemiMajor'][idx]
final['3FHL Assoc'] = Column(['']*len(final), dtype = 'U18')
final['3FHL Assoc'][mask] = fhl3['Source_Name'][idx[mask]]
hdu_list = fits.open('./catalogs/gammaray/4FGL_DR4.fits')
fgl4 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fgl4['RAJ2000']*u.degree, dec = fgl4['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fgl4['Conf_95_SemiMajor'][idx]
final['4FGL Assoc'] = Column(['']*len(final), dtype = 'U18')
final['4FGL Assoc'][mask] = fgl4['Source_Name'][idx[mask]]
final['4FGL Assoc'] = [name.replace(' ', '') for name in final['4FGL Assoc']]
final['4FGL Assoc'] = [name.replace('LJ', 'L J') for name in final['4FGL Assoc']]
hdu_list = fits.open('./catalogs/gammaray/4LAC-DR3.fits')
lac = Table(hdu_list[1].data)
catalog = SkyCoord(ra = lac['RACdeg']*u.degree, dec = lac['DECdeg']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
final['4LAC Assoc'] = Column(['']*len(final), dtype = 'U28')
final['4LAC Assoc'][mask] = lac['Assoc1'][idx[mask]]
hdu_list = fits.open('./catalogs/gammaray/1CGH_Preliminary_V2.0.fits')
cgh = Table(hdu_list[1].data)
catalog = SkyCoord(ra = cgh['RA_1CGH']*u.degree, dec = cgh['DEC_1CGH']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5.0
final['1CGH Assoc'] = Column(['']*len(final), dtype = 'U28')
final['1CGH Assoc'][mask] = cgh['Counterpart_name'][idx[mask]]


/var/folders/2z/y_y_51l51r1cdd9b0xsl_qj80000gn/T/ipykernel_39064/3548270837.py:38: StringTruncateWarning: truncated right side string(s) longer than 28 character(s) during assignment
  final['1CGH Assoc'][mask] = cgh['Counterpart_name'][idx[mask]]


In [234]:
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/bzcat.fits')
bzcat = Table(hdu_list[1].data)
catalog = SkyCoord(ra = bzcat['RA']*u.degree, dec = bzcat['DEC']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['BZCAT Assoc'] = Column(['']*len(final), dtype = 'U15')
final['BZCAT OTYPE'] = Column(['']*len(final), dtype = 'U24')
for i in range(len(final)): 
    if(mask[i]): 
        name = bzcat['NAME'][idx[i]]
        name = name[10:]
        final['BZCAT Assoc'][i] = name
        final['BZCAT OTYPE'][i] = bzcat['OBJECT_TYPE'][idx[i]]
        final['BZCAT OTYPE'][i] = final['BZCAT OTYPE'][i].replace('  ', '')
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/3HSP.fits')
hsp3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = hsp3['RAJ2000']*u.degree, dec = hsp3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['3HSP Assoc'] = Column(['']*len(final), dtype = 'U20')
final['3HSP Assoc'][mask] = hsp3['Name'][idx[mask]]
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/1whsp.fits')
whsp1 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp1['_RA']*u.degree, dec = whsp1['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['1WHSP Assoc'] = Column(['']*len(final), dtype = 'U15')
final['1WHSP Assoc'][mask] = whsp1['_1WHSP'][idx[mask]]
hdu_list= fits.open('./catalogs/other_blazar-hbl_cats/whsp2.fits')
whsp2 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp2['_RA']*u.degree, dec = whsp2['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
final['2WHSP Assoc'] = Column(['']*len(final), dtype = 'U15')
final['2WHSP Assoc'][mask] = whsp2['_2WHSPJ'][idx[mask]]

In [235]:
C20 = ascii.read('./catalogs/other_blazar-hbl_cats/C20.txt')
C20 = C20[0][:]
final['C20'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['BZCAT Assoc'][i] 
    target = target.replace(" ", "")
    if(target != "None"): 
        target1 = target[0:9]
        for j in range(len(C20)): 
            c_t = C20[j]
            c_t1 = c_t[0:9]
            if(target1 == c_t1): 
                target2 = target[10:14]
                c_t2 = c_t[10:14]
                if(target2 == c_t2): 
                    final['C20'][i] = 'Y'

hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot3.fits')
dabruscot3 = Table(hdu_list[1].data)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot5.fits')
dabruscot5 = Table(hdu_list[1].data)
final['D19'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['name'][i]
    target1 = target[0:10]
    for j in range(len(dabruscot3)): 
        catalog = dabruscot3['WISE'][j]
        catalog1 = catalog[0:10]
        if(target1 == catalog1): 
           target2 = target[11:21]
           catalog2 = catalog[11:21]
           if(target2 == catalog2): 
               final['D19'][i] = 'Y'
    for k in range(len(dabruscot5)): 
        catalog = dabruscot5['WISE'][k]
        catalog1 = catalog[0:10]
        if(target1 == catalog1): 
            target2 = target[11:21]
            catalog2 = catalog[11:21]
            if(target2 == catalog2): 
                final['D19'][i] = 'Y'

In [236]:
L25 = ascii.read('./catalogs/other_blazar-hbl_cats/L25.txt')
L25 = L25[0][:]
final['L25'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['4FGL Assoc'][i] 
    target1 = target[5:12]
    for j in range(len(L25)): 
        l_t = L25[j]
        l_t1 = l_t[0:7]
        if(target1 == l_t1): 
            target2 =target[13:17]
            l_t2 = l_t[8:12]
            if(target2 == l_t2): 
                final['L25'][i] = 'Y'
                
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/Marchesi_Xray_Bl_AA_25.fits')
marchesi = Table(hdu_list[1].data)
final['MAR25'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    target = final['BZCAT Assoc'][i]
    if(target != "None"):
        target = target.replace(" ", "")
        target1 = target[0:9]
        for j in range(len(marchesi)): 
            catalog = marchesi['ID_5BZCAT'][j]
            catalog1 = catalog[0:9]
            if(target1 == catalog1): 
                target2 = target[10:14]
                catalog2 = catalog[10:14]
                if(target2 == catalog2): 
                    final['MAR25'][i] = 'Y'
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/massaro_selection.fits')
massaro = Table(hdu_list[1].data)
final['MAS13'] = Column(['N']*len(final), dtype = 'U1')
for i in range(len(final)): 
    name = final['name'][i]
    if(name in massaro['WISE']): 
        final['MAS13'][i] = 'Y'
        idx = np.where(massaro['WISE'] == name)[0][0]


In [237]:
final.sort('ra')

In [238]:
final.rename_column('name', 'WISEA')

In [239]:
final.rename_column('ra', 'RA')
final.rename_column('dec', 'DEC')

In [241]:
final

THC,WISEA,1eRASS,RA,DEC,EXT_LIKE,w2 - w3,w1 - w2,dist,Alpha IRX,TeVCAT Assoc,2FHL Assoc,3FHL Assoc,4FGL Assoc,4LAC Assoc,1CGH Assoc,BZCAT Assoc,BZCAT OTYPE,3HSP Assoc,1WHSP Assoc,2WHSP Assoc,C20,D19,L25,MAR25,MAS13
str9,str19,str16,float64,float64,float32,float64,float64,float64,float64,str21,str17,str18,str18,str28,str28,str15,str24,str20,str15,str15,str1,str1,str1,str1,str1
,J000027.26-750808.8,J000026.5-750806,0.1136099,-75.1357968,0.0,2.1839999999999993,0.8150000000000013,0.023428284674258962,1.1394589577747862,,,,,,,,,,,,N,N,N,N,N
,J000038.66-463903.5,J000038.5-463903,0.161114,-46.6509755,0.0,2.0860000000000003,0.17600000000000016,0.40336722647643186,1.3941920233950544,,,,,,,,,,,,N,N,N,N,N
,J000126.45-501501.6,J000126.3-501505,0.3602134,-50.2504621,0.0,2.158999999999999,0.5210000000000008,0.1858417415014598,1.3601995464764045,,,,,,,,,,,,N,N,N,N,N
J0001-419,J000132.74-415525.2,J000132.7-415526,0.3864515,-41.9236926,0.0,2.1709999999999994,0.6799999999999997,0.07168848188704327,0.7223740485184954,,,3FHL J0001.9-4155,4FGL J0001.6-4156,2MASS J00013275-4155252,3HSPJ000132.7-415525,,,3HSPJ000132.7-415525,000132.7-415525,000132.6-415524,N,Y,N,N,N
,J000347.32-501150.4,J000347.3-501149,0.9471974,-50.19736,0.0,1.2729999999999997,0.0860000000000003,0.04987380269145551,1.345478674211382,,,,,,,,,,,,N,N,N,N,N
,J000434.22-400035.1,J000434.0-400038,1.1426222,-40.0097747,0.0,2.378,0.48499999999999943,0.3541045477973454,1.2987618872203737,,,,4FGL J0004.4-4001,,,,,,,,N,Y,N,N,N
,J000448.48-634813.5,J000448.3-634815,1.2020243,-63.8037732,0.0,2.337999999999999,0.8659999999999997,0.03640386605255049,1.0632802856837822,,,,,,,,,,,,N,N,N,N,N
,J000450.84-691150.7,J000450.0-691149,1.2118644,-69.1974304,4.017794,1.6069999999999993,0.3310000000000013,0.02310728629784535,1.2820059790109375,,,,,,,,,,,,N,N,N,N,N
,J000504.33-414829.8,J000504.6-414832,1.2680801,-41.8082989,0.0,1.5439999999999987,0.36400000000000077,0.08886641648213993,1.278675630175073,,,,,,,,,,,,N,N,N,N,N


In [30]:
hdu_list = fits.open('./catalogs/WISE_EROSITA_ALLmatches.fits')
wise_erosita = Table(hdu_list[1].data)

In [ ]:
hdu_list = fits.open('./catalogs/xray/eRASS1_Main.v1.1.fits')
erass1 = Table(hdu_list[1].data)

In [40]:
ext_like = [erass1['EXT_LIKE'][i] for i, name in enumerate(erass1['IAUNAME']) if name in wise_erosita['eROSITA Name']]

In [71]:
wise_erosita['EXT_LIKE'] = ext_like

In [75]:
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot3.fits')
dabruscot3 = Table(hdu_list[1].data)
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/dabruscot5.fits')
dabruscot5 = Table(hdu_list[1].data)

In [85]:
wise_erosita['D19'] = ['N'] *len(wise_erosita)

In [86]:
for i in range(len(wise_erosita)): 
    if wise_erosita['WISE Name'][i] in dabruscot3['WISE']:
        wise_erosita['D19'][i] = 'Y'
    if wise_erosita['WISE Name'][i] in dabruscot5['WISE']:
        wise_erosita['D19'][i] = 'Y'

In [98]:
hdu_list = fits.open('./catalogs/gammaray/2FHL.fits')
fhl = Table(hdu_list[1].data)

In [99]:
fhl

Source_Name,RAJ2000,DEJ2000,GLON,GLAT,Pos_err_68,Pos_err_95,TS,Spectral_Index,Unc_Spectral_Index,Intr_Spectral_Index_D11,Unc_Intr_Spectral_Index_D11,Intr_Spectral_Index_G12,Unc_Intr_Spectral_Index_G12,Flux50,Unc_Flux50,Energy_Flux50,Unc_Energy_Flux50,Flux50_171GeV,Unc_Flux50_171GeV,Sqrt_TS50_171GeV,Flux171_585GeV,Unc_Flux171_585GeV,Sqrt_TS171_585GeV,Flux585_2000GeV,Unc_Flux585_2000GeV,Sqrt_TS585_2000GeV,Npred,HEP_Energy,HEP_Prob,ROI,ASSOC,ASSOC_PROB_BAY,ASSOC_PROB_LR,CLASS,Redshift,NuPeak_obs,3FGL_Name,1FHL_Name,TeVCat_Name
str18,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32[2],float32,float32,float32[2],float32,float32,float32[2],float32,float32,float32,float32,int16,str25,float32,float32,str8,float32,float32,str18,str18,str18
2FHL J0008.1+4709,2.0437,47.1642,115.339355,-15.068757,0.06111402,0.098393574,28.64,6.24,2.75,3.96,3.19,2.16,4.21,1.23e-11,6.71e-12,1.21e-12,6.71e-13,3.3634447e-12,-1.5366806e-12 .. 2.1609453e-12,5.353536,4.3494674e-18,nan .. 4.7538935e-12,0.0,8.448714e-18,nan .. 7.291014e-12,0.0,4.0,68.15,0.99,1,MG4 J000800+4712,0.99721974,0.8348271,bll,2.1,2511884200000000.0,3FGL J0008.0+4713,1FHL J0007.7+4709,
2FHL J0009.3+5031,2.3435,50.5217,116.12411,-11.793202,0.045439195,0.0731571,53.97,5.08,1.66,nan,nan,nan,nan,1.91e-11,7.82e-12,2.03e-12,8.79e-13,8.362822e-12,-2.9896227e-12 .. 3.895122e-12,7.351317,2.9145808e-17,nan .. 5.1000337e-12,0.0,3.5087458e-16,nan .. 4.874578e-12,0.0,6.4,72.76,1.0,1,NVSS J000922+503028,0.99972373,0.7348077,bll,0.0,1412536400000000.0,3FGL J0009.3+5030,1FHL J0009.2+5032,
2FHL J0018.5+2947,4.6355,29.7879,114.46349,-32.54235,0.037093636,0.059720755,30.89,2.58,0.99,2.41,1.04,2.4,1.04,1.06e-11,6.15e-12,2.05e-12,1.72e-12,9.65438e-12,-5.5534162e-12 .. 5.5534162e-12,5.765786,1.1790023e-15,nan .. 5.378652e-12,0.0,1.6046534e-16,nan .. 6.120081e-12,0.0,3.0,127.32,1.0,3,RBS 0042,0.9998682,0.97852194,bll,0.1,5.9156075e+16,3FGL J0018.4+2947,1FHL J0018.6+2946,
2FHL J0022.0+0006,5.5001,0.1059,107.171715,-61.86175,0.05118522,0.082408205,29.96,1.86,0.57,0.95,0.72,0.88,0.71,1.97e-11,9.56e-12,6.86e-12,5.29e-12,1.6119727e-11,-7.1725507e-12 .. 9.9534886e-12,5.3821964,3.6392152e-12,-4.3863624e-12 .. 4.3863624e-12,1.3853942,8.423558e-16,nan .. 7.342399e-12,0.0,4.8,180.13,0.86,2,5BZGJ0022+0006,0.99928,0.90008944,bll-g,0.306,4.315193e+16,,,
2FHL J0033.6-1921,8.4115,-19.3575,94.28002,-81.22237,0.034838412,0.056089845,148.31,3.32,0.69,2.56,0.88,2.33,0.92,5.46e-11,1.5e-11,7.62e-12,2.69e-12,4.0016098e-11,-1.0161534e-11 .. 1.223782e-11,12.172502,2.1390131e-12,nan .. 8.258118e-12,0.9079581,2.439548e-17,nan .. 6.8422585e-12,0.0,13.8,170.01,0.99,2,KUV 00311-1938,0.999759,0.9814242,bll,0.61,8317639000000000.0,3FGL J0033.6-1921,1FHL J0033.6-1921,TeV J0033-193
2FHL J0035.8+5949,8.9625,59.8312,120.97197,-2.9812155,0.031602595,0.05088018,402.4,2.23,0.21,nan,nan,nan,nan,1.25e-10,1.9e-11,3.11e-11,7.23e-12,9.4229395e-11,-1.5256577e-11 .. 1.7157154e-11,17.312449,2.6699127e-11,-7.628179e-12 .. 9.450845e-12,10.390878,3.7260905e-16,nan .. 4.6744392e-12,0.0,46.5,247.62,0.96,4,1ES 0033+595,0.99995834,0.9928678,bll,0.0,1.3182593e+17,3FGL J0035.9+5949,1FHL J0035.9+5950,TeV J0035+598
2FHL J0040.3+4049,10.0949,40.8315,120.676285,-21.991812,0.035514843,0.057178896,26.76,2.12,0.81,nan,nan,nan,nan,1.05e-11,6.3e-12,2.84e-12,2.67e-12,7.415768e-12,-4.077804e-12 .. 6.4518156e-12,4.954423,3.0030787e-12,-2.317754e-12 .. 4.466438e-12,1.7083547,4.7316626e-16,nan .. 5.7980946e-12,0.0,3.2,258.77,0.85,3,B3 0037+405,0.9987379,0.9346999,bcu I,0.0,1.0,3FGL J0040.3+4049,1FHL J0040.3+4049,
2FHL J0043.9+3424,10.9755,34.4109,121.16442,-28.434984,0.059241667,0.095379084,39.5,4.57,1.61,3.46,1.85,2.95,1.91,1.83e-11,8.24e-12,2.03e-12,9.93e-13,9.657617e-12,-4.316866e-12 .. 4.316866e-12,6.3115244,8.758609e-17,nan .. 5.2433453e-12,0.0,9.438122e-16,nan .. 5.687869e-12,0.0,5.4,109.97,0.9,3,GB6 J0043+3426,0.99859357,0.8388443,fsrq,0.966,64565484000000.0,3FGL J0043.8+

In [95]:
final = Table([wise_erosita['w1mpro'], wise_erosita['w1sigmpro'], wise_erosita['w2mpro'], wise_erosita['w2sigmpro'], wise_erosita['w3mpro'], wise_erosita['w3sigmpro'], wise_erosita['dist'], wise_erosita['Alpha_IRX'], wise_erosita['EXT_LIKE'], wise_erosita['2FHL Assoc'], wise_erosita['3FHL Assoc'], wise_erosita['4FGL Assoc'], wise_erosita['4LAC-DR4 Assoc'], wise_erosita['BZCAT Assoc'], wise_erosita['BZCAT OTYPE'], wise_erosita['3HSP Assoc'], wise_erosita['1WHSP Assoc'], wise_erosita['2WHSP Assoc'], wise_erosita['C20'], wise_erosita['D19'], wise_erosita['L25'], wise_erosita['MAR25'], wise_erosita['MAS13']])

In [207]:
wise_erosita = final

In [208]:
wise_erosita

THC,WISEA,1eRASS,RA,DEC,EXT_LIKE,w2 - w3,w1 - w2,dist,Alpha IRX,TeVCAT Assoc,2FHL Assoc,3FHL Assoc,4FGL Assoc,4LAC Assoc,1CGH Assoc,BZCAT Assoc,BZCAT OTYPE,3HSP Assoc,1WHSP Assoc,2WHSP Assoc,C20,D19,L25,MAR25,MAS13
str9,str19,str16,float64,float64,float32,float64,float64,float64,float64,str21,str17,str18,str18,str28,str28,str15,str24,str20,str15,str15,str1,str1,str1,str1,str1
,J000027.26-750808.8,J000026.5-750806,0.1136099,-75.1357968,0.0,2.1839999999999993,0.8150000000000013,0.023428284674258962,1.1394589577747862,,,,,,,,,,,,N,N,N,N,N
,J000038.66-463903.5,J000038.5-463903,0.161114,-46.6509755,0.0,2.0860000000000003,0.17600000000000016,0.40336722647643186,1.3941920233950544,,,,,,,,,,,,N,N,N,N,N
,J000126.45-501501.6,J000126.3-501505,0.3602134,-50.2504621,0.0,2.158999999999999,0.5210000000000008,0.1858417415014598,1.3601995464764045,,,,,,,,,,,,N,N,N,N,N
J0001-419,J000132.74-415525.2,J000132.7-415526,0.3864515,-41.9236926,0.0,2.1709999999999994,0.6799999999999997,0.07168848188704327,0.7223740485184954,,,3FHL J0001.9-4155,4FGL J0001.6-4156,2MASS J00013275-4155252,3HSPJ000132.7-415525,,,3HSPJ000132.7-415525,000132.7-415525,000132.6-415524,N,Y,N,N,N
,J000347.32-501150.4,J000347.3-501149,0.9471974,-50.19736,0.0,1.2729999999999997,0.0860000000000003,0.04987380269145551,1.345478674211382,,,,,,,,,,,,N,N,N,N,N
,J000434.22-400035.1,J000434.0-400038,1.1426222,-40.0097747,0.0,2.378,0.48499999999999943,0.3541045477973454,1.2987618872203737,,,,4FGL J0004.4-4001,,,,,,,,N,Y,N,N,N
,J000448.48-634813.5,J000448.3-634815,1.2020243,-63.8037732,0.0,2.337999999999999,0.8659999999999997,0.03640386605255049,1.0632802856837822,,,,,,,,,,,,N,N,N,N,N
,J000450.84-691150.7,J000450.0-691149,1.2118644,-69.1974304,4.017794,1.6069999999999993,0.3310000000000013,0.02310728629784535,1.2820059790109375,,,,,,,,,,,,N,N,N,N,N
,J000504.33-414829.8,J000504.6-414832,1.2680801,-41.8082989,0.0,1.5439999999999987,0.36400000000000077,0.08886641648213993,1.278675630175073,,,,,,,,,,,,N,N,N,N,N


In [ ]:
tbl= QTable(wise_erosita)
tbl['w2mpro-w3mpro'] = tbl['w2mpro-w3mpro']*u.mag
tbl['w2mpro-w3mpro_ERR'] = tbl['w2mpro-w3mpro_ERR']*u.mag
tbl['w1mpro-w2mpro'] = tbl['w1mpro-w2mpro']*u.mag
tbl['w1mpro-w2mpro_ERR'] = tbl['w1mpro-w2mpro_ERR']*u.mag
table_hdu = fits.BinTableHDU(tbl) 
header = table_hdu.header
header['EXTNAME'] = 'ALL_eROSITA_WISE_MATCHES'
header['AUTHOR'] = 'Cassie Metzger' 
header['DATE'] = '2025-04-16' 
header['WAVELENG'] = 'Multi'
header['DATASRC'] = 'eROSITA, WISE'
header['DOI'] = '10.48550/arXiv.2501.12520' 
header['WAVELENG'] = 'Multi'
header['SELPRO'] = 'All sources in the eROSITA catalog with WISE counterparts within 5'' that have IR magnitudes less than 14.3, 13.8, 12.2 in the W1, W2, W3 bands respectively and IR colors than fall between 1.25-2.4 and 0.05-0.9 in the W2-W3 and W1-W2 bands,respectively.'
tbl.write('./catalogs/WISE_EROSITA_ALLmatches.fits', format = 'fits', overwrite = True)
print('Catalog saved as WISE_EROSITA_ALLmatches.fits')

w1mpro,w1sigmpro,w2mpro,w2sigmpro,w3mpro,w3sigmpro,dist,Alpha_IRX,EXT_LIKE,2FHL Assoc,3FHL Assoc,4FGL Assoc,4LAC-DR4 Assoc,BZCAT Assoc,BZCAT OTYPE,3HSP Assoc,1WHSP Assoc,2WHSP Assoc,C20,D19,L25,MAR25,MAS13
float64,float64,float64,float64,float64,float64,float64,float64,float32,str17,str18,str18,str18,str15,str24,str20,str15,str15,str1,str1,str1,str1,str1
13.855,0.025,13.04,0.026,10.856,0.1,0.023,1.139,0.0,,,,,,,,,,N,N,N,N,N
13.476,0.026,13.3,0.028,11.214,0.135,0.403,1.394,0.0,,,,,,,,,,N,N,N,N,N
13.974,0.029,13.453,0.031,11.294,0.191,0.186,1.36,0.0,,,,,,,,,,N,N,N,N,N
13.988,0.026,13.308,0.031,11.137,0.116,0.072,0.722,0.0,,3FHL J0001.9-4155,4FGL J0001.6-4156,J0001.6-4156,,,3HSPJ000132.7-415525,000132.7-415525,000132.6-415524,N,Y,N,N,N
13.432,0.026,13.346,0.029,12.073,nan,0.05,1.345,0.0,,,,,,,,,,N,N,N,N,N
13.282,0.024,12.797,0.025,10.419,0.073,0.354,1.299,0.0,,,4FGL J0004.4-4001,,,,,,,N,Y,N,N,N
13.956,0.025,13.09,0.025,10.752,0.09,0.036,1.063,0.0,,,,,,,,,,N,N,N,N,N
13.887,0.025,13.556,0.029,11.949,0.243,0.023,1.282,0.0,,,,,,,,,,N,N,N,N,N
14.048,0.026,13.684,0.033,12.14,0.378,0.089,1.279,0.0,,,,,,,,,,N,N,N,N,N


In [3]:
hdu_list = fits.open('./catalogs/IR/wise.fits')
wise = Table(hdu_list[1].data)

In [4]:
def get_wise_flux(catalog): 
    sol = 2.997925E14 # variable sol; c in microns/s
    w1=sol/3.368
    w2=sol/4.618
    w3=sol/12.082

    fc_w1=0.9921
    fc_w2=0.9943
    fc_w3=0.9373

    F_w1 = 306.682 #Jy
    F_w2 = 170.663 #Jy
    F_w3 = 29.045 #Jy

    flux_w1 = F_w1/fc_w1*10**(-catalog['w1mpro']/2.5)
    flux_w2 = F_w2/fc_w2*10**(-catalog['w2mpro']/2.5)
    flux_w3 = F_w3/fc_w3*10**(-catalog['w3mpro']/2.5)

    #errors 
    w1errh = catalog['w1mpro'] - catalog['w1sigmpro']
    w2errh = catalog['w2mpro'] - catalog['w2sigmpro']
    w3errh = catalog['w3mpro'] - catalog['w3sigmpro']

    w1errl = catalog['w1mpro'] + catalog['w1sigmpro']
    w2errl = catalog['w2mpro'] + catalog['w2sigmpro']
    w3errl = catalog['w3mpro'] + catalog['w3sigmpro']

    #relative errors in Jy 

    err_w1_h = F_w1/fc_w1*10**(-w1errh/2.5)-flux_w1
    err_w2_h = F_w2/fc_w2*10**(-w2errh/2.5)-flux_w2
    err_w3_h = F_w3/fc_w3*10**(-w3errh/2.5)-flux_w3

    err_w1_l = F_w1/fc_w1*10**(-w1errl/2.5)-flux_w1
    err_w2_l = F_w2/fc_w2*10**(-w2errl/2.5)-flux_w2
    err_w3_l = F_w3/fc_w3*10**(-w3errl/2.5)-flux_w3

    err_w1 = 0.5*(np.abs(err_w1_h)+ np.abs(err_w1_l))
    err_w2 = 0.5*(np.abs(err_w2_h)+np.abs(err_w2_l))
    err_w3 = 0.5*(np.abs(err_w3_h)+np.abs(err_w3_l))

    flux = [flux_w1,flux_w2,flux_w3]
    err = [err_w1, err_w2, err_w3]
    catalog['w1flux'] = flux_w1 
    catalog['w2flux'] = flux_w2 
    catalog['w3flux'] = flux_w3
    catalog['w1sigflux'] = err_w1
    catalog['w2sigflux'] = err_w2
    catalog['w3sigflux'] = err_w3
    return catalog

In [7]:
m = 0.83809562126163
b = -1.045969175938493
leng = len(wise_erosita)
def distance (m,b, x,y): 
   return np.abs(m * x + b-y)/np.sqrt(m**2 + 1**2)
wise_erosita['dist'] = distance(m,b, wise_erosita['w2mpro-w3mpro'], wise_erosita['w1mpro-w2mpro'])

In [8]:
wise = get_wise_flux(wise)

In [9]:
#choosing erosita 0.2-2.3 keV energy band, Eband = 1 keV 
e_energy = 1.6022e-16 #J 
h = 6.626e-34 #Js
e_v1 = e_energy/h
w_v = 3E8/3.4E-6

In [10]:
wise_erosita['ML_FLUX_1_JY'] = 1E+23 * wise_erosita['ML_FLUX_1'] * 1/(e_v1)

In [13]:
wise_erosita

WISE Name,eROSITA Name,ra,dec,w1mpro,w1sigmpro,w2mpro,w2sigmpro,w3mpro,w3sigmpro,w1flux,w1sigflux,ML_FLUX_1,ML_FLUX_ERR_1,w2mpro-w3mpro,w2mpro-w3mpro_ERR,w1mpro-w2mpro,w1mpro-w2mpro_ERR,dist,Alpha_IRX,2FHL Assoc,3FHL Assoc,4FGL Assoc,4LAC-DR4 Assoc,BZCAT Assoc,BZCAT OTYPE,3HSP Assoc,1WHSP Assoc,2WHSP Assoc,C20,L25,MAR25,MAS13,ML_FLUX_1_JY
str19,str23,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str17,str18,str18,str18,str15,str24,str20,str15,str15,str1,str1,str1,str1,float64
J000027.26-750808.8,1eRASS J000026.5-750806,0.11361,-75.1358,13.855,0.025,13.04,0.026,10.856,0.1,0.887,0.02,2.596e-13,4.162e-14,2.184,0.103,0.815,0.036,0.023428284674257432,1.139,,,,,,,,,,N,N,N,N,1.0735923105729621e-07
J000038.66-463903.5,1eRASS J000038.5-463903,0.16111,-46.65098,13.476,0.026,13.3,0.028,11.214,0.135,1.258,0.03,4.9e-14,2.405e-14,2.086,0.138,0.176,0.038,0.4033672264764317,1.394,,,,,,,,,,N,N,N,N,2.0264261640244662e-08
J000126.45-501501.6,1eRASS J000126.3-501505,0.36021,-50.25046,13.974,0.029,13.453,0.031,11.294,0.191,0.795,0.021,4.054e-14,1.931e-14,2.159,0.193,0.521,0.042,0.1858417415014609,1.36,,,,,,,,,,N,N,N,N,1.676557483460242e-08
J000132.74-415525.2,1eRASS J000132.7-415526,0.38645,-41.92369,13.988,0.026,13.308,0.031,11.137,0.116,0.785,0.019,6.237e-12,2.528e-13,2.171,0.12,0.68,0.04,0.07168848188704335,0.722,,3FHL J0001.9-4155,4FGL J0001.6-4156,J0001.6-4156,,,3HSPJ000132.7-415525,000132.7-415525,000132.6-415524,N,N,N,N,2.579351017351142e-06
J000347.32-501150.4,1eRASS J000347.3-501149,0.9472,-50.19736,13.432,0.026,13.346,0.029,12.073,nan,1.31,0.031,7.504e-14,2.752e-14,1.273,nan,0.086,0.039,0.049873802691455284,1.345,,,,,,,,,,N,N,N,N,3.103326925477468e-08
J000434.22-400035.1,1eRASS J000434.0-400038,1.14262,-40.00977,13.282,0.024,12.797,0.025,10.419,0.073,1.504,0.033,1.247e-13,3.736e-14,2.378,0.077,0.485,0.035,0.35410454779734496,1.299,,,4FGL J0004.4-4001,,,,,,,N,N,N,N,5.1570478092622634e-08
J000448.48-634813.5,1eRASS J000448.3-634815,1.20202,-63.80377,13.956,0.025,13.09,0.025,10.752,0.09,0.809,0.019,4.323e-13,5.627e-14,2.338,0.093,0.866,0.035,0.03640386605255074,1.063,,,,,,,,,,N,N,N,N,1.787804144301585e-07
J000450.84-691150.7,1eRASS J000450.0-691149,1.21186,-69.19743,13.887,0.025,13.556,0.029,11.949,0.243,0.862,0.02,8.156e-14,2.814e-14,1.607,0.245,0.331,0.038,0.023107286297843862,1.282,,,,,,,,,,N,N,N,N,3.372965672200724e-08
J000504.33-414829.8,1eRASS J000504.6-414832,1.26808,-41.8083,14.048,0.026,13.684,0.033,12.14,0.378,0.743,0.018,7.22e-14,2.809e-14,1.544,0.379,0.364,0.042,0.08886641648213847,1.279,,,,,,,,,,N,N,N,N,2.98587691923605e-08


In [14]:
wise_erosita['Alpha_IRX'] = -np.log10(wise_erosita['w1flux']/wise_erosita['ML_FLUX_1_JY'])/np.log10(w_v/e_v1)

In [241]:
wise_erosita = Table([wise['name'][wise_erosita['WISE idx']], wise_erosita['IAUNAME'], wise['ra'][wise_erosita['WISE idx']], wise['dec'][wise_erosita['WISE idx']], wise['w1mpro'][wise_erosita['WISE idx']], wise['w1sigmpro'][wise_erosita['WISE idx']], wise['w2mpro'][wise_erosita['WISE idx']], wise['w2sigmpro'][wise_erosita['WISE idx']], wise['w3mpro'][wise_erosita['WISE idx']], wise['w3sigmpro'][wise_erosita['WISE idx']], wise['w1flux'][wise_erosita['WISE idx']], wise['w1sigflux'][wise_erosita['WISE idx']], wise_erosita['ML_FLUX_1'], wise_erosita['ML_FLUX_ERR_1'], wise_erosita['x'], wise_erosita['x_err'], wise_erosita['y'], wise_erosita['y_err'], wise_erosita['dist'], wise_erosita['Alpha_IRX']])

In [16]:
hdu_list = fits.open('./catalogs/gammaray/2FHL.fits')
fhl2 = Table(hdu_list[1].data)
c = SkyCoord(ra = wise_erosita['ra']*u.degree, dec = wise_erosita['dec']*u.degree, frame = 'fk5')
catalog = SkyCoord(ra = fhl2['RAJ2000']*u.degree, dec = fhl2['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl2['Pos_err_95'][idx]
wise_erosita['2FHL Assoc'] = Column(['']*len(wise_erosita), dtype = 'U17')
wise_erosita['2FHL Assoc'][mask] = fhl2['Source_Name'][idx[mask]]
hdu_list = fits.open('./catalogs/gammaray/3FHL.fits')
fhl3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fhl3['RAJ2000']*u.degree, dec = fhl3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fhl3['Conf_95_SemiMajor'][idx]
wise_erosita['3FHL Assoc'] = Column(['']*len(wise_erosita), dtype = 'U18')
wise_erosita['3FHL Assoc'][mask] = fhl3['Source_Name'][idx[mask]]
hdu_list = fits.open('./catalogs/gammaray/4FGL_DR4.fits')
fgl4 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = fgl4['RAJ2000']*u.degree, dec = fgl4['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.deg <= fgl4['Conf_95_SemiMajor'][idx]
wise_erosita['4FGL Assoc'] = Column(['']*len(wise_erosita), dtype = 'U18')
wise_erosita['4FGL Assoc'][mask] = fgl4['Source_Name'][idx[mask]]
wise_erosita['4FGL Assoc'] = [name.replace(' ', '') for name in wise_erosita['4FGL Assoc']]
wise_erosita['4FGL Assoc'] = [name.replace('LJ', 'L J') for name in wise_erosita['4FGL Assoc']]
hdu_list = fits.open('./catalogs/gammaray/4LAC-DR3.fits')
lac = Table(hdu_list[1].data)
lac['_4FGL'] = [lac['_4FGL'][i].replace(' ', '') for i in range(len(lac))]
wise_erosita['4LAC-DR4 Assoc'] = Column(['']*len(wise_erosita), dtype = 'U18')
catalog = SkyCoord(ra = lac['RACdeg']*u.degree, dec = lac['DECdeg']*u.degree, frame = 'fk5')
idx,d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <=5 
wise_erosita['4LAC-DR4 Assoc'][mask] = lac['_4FGL'][idx[mask]]
        

In [17]:
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/bzcat.fits')
bzcat = Table(hdu_list[1].data)
catalog = SkyCoord(ra = bzcat['RA']*u.degree, dec = bzcat['DEC']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
wise_erosita['BZCAT Assoc'] = Column(['']*len(wise_erosita), dtype = 'U15')
wise_erosita['BZCAT OTYPE'] = Column(['']*len(wise_erosita), dtype = 'U24')
for i in range(len(wise_erosita)): 
    if(mask[i]): 
        name = bzcat['NAME'][idx[i]]
        name = name[10:]
        wise_erosita['BZCAT Assoc'][i] = name
        wise_erosita['BZCAT OTYPE'][i] = bzcat['OBJECT_TYPE'][idx[i]]
        wise_erosita['BZCAT OTYPE'][i] = wise_erosita['BZCAT OTYPE'][i].replace('  ', '')
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/3HSP.fits')
hsp3 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = hsp3['RAJ2000']*u.degree, dec = hsp3['DEJ2000']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
wise_erosita['3HSP Assoc'] = Column(['']*len(wise_erosita), dtype = 'U20')
wise_erosita['3HSP Assoc'][mask] = hsp3['Name'][idx[mask]]
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/1whsp.fits')
whsp1 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp1['_RA']*u.degree, dec = whsp1['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
wise_erosita['1WHSP Assoc'] = Column(['']*len(wise_erosita), dtype = 'U15')
wise_erosita['1WHSP Assoc'][mask] = whsp1['_1WHSP'][idx[mask]]
hdu_list= fits.open('./catalogs/other_blazar-hbl_cats/whsp2.fits')
whsp2 = Table(hdu_list[1].data)
catalog = SkyCoord(ra = whsp2['_RA']*u.degree, dec = whsp2['_DE']*u.degree, frame = 'fk5')
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
mask = d2d.arcsec <= 5
wise_erosita['2WHSP Assoc'] = Column(['']*len(wise_erosita), dtype = 'U15')
wise_erosita['2WHSP Assoc'][mask] = whsp2['_2WHSPJ'][idx[mask]]

In [18]:
C20 = ascii.read('./catalogs/other_blazar-hbl_cats/C20.txt')
C20 = C20[0][:]

In [19]:
wise_erosita['C20'] = Column(['N']*len(wise_erosita), dtype = 'U1')
x = 0 
for i in range(len(wise_erosita)): 
    target = wise_erosita['BZCAT Assoc'][i] 
    target = target.replace(" ", "")
    if(target != ''): 
        target1 = target[0:9]
        for j in range(len(C20)): 
            c_t = C20[j]
            c_t1 = c_t[0:9]
            if(target1 == c_t1): 
                target2 = target[10:14]
                c_t2 = c_t[10:14]
                if(target2 == c_t2): 
                    wise_erosita['C20'][i] = 'Y'
                    x+=1

In [20]:
L25 = ascii.read('./catalogs/other_blazar-hbl_cats/L25.txt')
L25 = L25[0][:]

In [24]:
wise_erosita['L25'] = Column(['N']*len(wise_erosita), dtype = 'U1')
for i in range(len(wise_erosita)): 
    target = wise_erosita['4FGL Assoc'][i] 
    target1 = target[5:12]
    for j in range(len(L25)): 
        l_t = L25[j]
        l_t1 = l_t[0:7]
        if(target1 == l_t1): 
            target2 =target[13:17]
            l_t2 = l_t[8:12]
            if(target2 == l_t2): 
                wise_erosita['L25'][i] = 'Y'
                
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/Marchesi_Xray_Bl_AA_25.fits')
marchesi = Table(hdu_list[1].data)
wise_erosita['MAR25'] = Column(['N']*len(wise_erosita), dtype = 'U1')
for i in range(len(wise_erosita)): 
    target = wise_erosita['BZCAT Assoc'][i]
    if(target != "None"):
        target = target.replace(" ", "")
        target1 = target[0:9]
        for j in range(len(marchesi)): 
            catalog = marchesi['ID_5BZCAT'][j]
            catalog1 = catalog[0:9]
            if(target1 == catalog1): 
                target2 = target[10:14]
                catalog2 = catalog[10:14]
                if(target2 == catalog2): 
                    wise_erosita['MAR25'][i] = 'Y'
hdu_list = fits.open('./catalogs/other_blazar-hbl_cats/massaro_selection.fits')
massaro = Table(hdu_list[1].data)
wise_erosita['MAS13'] = Column(['N']*len(wise_erosita), dtype = 'U1')
for i in range(len(wise_erosita)): 
    name = wise_erosita['WISE Name'][i]
    if(name in massaro['WISE']): 
        wise_erosita['MAS13'][i] = 'Y'
        idx = np.where(massaro['WISE'] == name)[0][0]


In [25]:
import math
from decimal import Decimal, ROUND_HALF_UP, getcontext

def format_sigfigs(num, sigfig):
    if num is None or (isinstance(num, float) and math.isnan(num)):
        return "nan"

    # Ensure context precision is high enough
    getcontext().prec = 10

    # Convert to Decimal for consistent rounding
    d = Decimal(str(num))
    rounded = d.quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)

    # Format with exactly 2 digits after the decimal point
    return f"{rounded:.{sigfig}f}"


In [26]:
wise_erosita['ra'] = [format_sigfigs(ra, 2) for ra in wise_erosita['ra']]
wise_erosita['dec'] = [format_sigfigs(dec, 2) for dec in wise_erosita['dec']]
wise_erosita['w1mpro']

WISE Name,eROSITA Name,ra,dec,w1mpro,w1sigmpro,w2mpro,w2sigmpro,w3mpro,w3sigmpro,w1flux,w1sigflux,ML_FLUX_1,ML_FLUX_ERR_1,w2mpro-w3mpro,w2mpro-w3mpro_ERR,w1mpro-w2mpro,w1mpro-w2mpro_ERR,dist,Alpha_IRX,2FHL Assoc,3FHL Assoc,4FGL Assoc,4LAC-DR4 Assoc,BZCAT Assoc,BZCAT OTYPE,3HSP Assoc,1WHSP Assoc,2WHSP Assoc,C20,L25,MAR25,MAS13,ML_FLUX_1_JY
str19,str23,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str17,str18,str18,str18,str15,str24,str20,str15,str15,str1,str1,str1,str1,float64
J000027.26-750808.8,1eRASS J000026.5-750806,0.11361,-75.1358,13.855,0.025,13.04,0.026,10.856,0.1,0.887,0.02,2.596e-13,4.162e-14,2.184,0.103,0.815,0.036,0.023428284674257432,2.0120536489830294,,,,,,,,,,N,N,N,N,1.0735923105729621e-07
J000038.66-463903.5,1eRASS J000038.5-463903,0.16111,-46.65098,13.476,0.026,13.3,0.028,11.214,0.135,1.258,0.03,4.9e-14,2.405e-14,2.086,0.138,0.176,0.038,0.4033672264764317,2.266826986654193,,,,,,,,,,N,N,N,N,2.0264261640244662e-08
J000126.45-501501.6,1eRASS J000126.3-501505,0.36021,-50.25046,13.974,0.029,13.453,0.031,11.294,0.191,0.795,0.021,4.054e-14,1.931e-14,2.159,0.193,0.521,0.042,0.1858417415014609,2.2327934781796044,,,,,,,,,,N,N,N,N,1.676557483460242e-08
J000132.74-415525.2,1eRASS J000132.7-415526,0.38645,-41.92369,13.988,0.026,13.308,0.031,11.137,0.116,0.785,0.019,6.237e-12,2.528e-13,2.171,0.12,0.68,0.04,0.07168848188704335,1.5950091890293705,,3FHL J0001.9-4155,4FGL J0001.6-4156,J0001.6-4156,,,3HSPJ000132.7-415525,000132.7-415525,000132.6-415524,N,N,N,N,2.579351017351142e-06
J000347.32-501150.4,1eRASS J000347.3-501149,0.9472,-50.19736,13.432,0.026,13.346,0.029,12.073,nan,1.31,0.031,7.504e-14,2.752e-14,1.273,nan,0.086,0.039,0.049873802691455284,2.218102519794271,,,,,,,,,,N,N,N,N,3.103326925477468e-08
J000434.22-400035.1,1eRASS J000434.0-400038,1.14262,-40.00977,13.282,0.024,12.797,0.025,10.419,0.073,1.504,0.033,1.247e-13,3.736e-14,2.378,0.077,0.485,0.035,0.35410454779734496,2.171387755985483,,,4FGL J0004.4-4001,,,,,,,N,N,N,N,5.1570478092622634e-08
J000448.48-634813.5,1eRASS J000448.3-634815,1.20202,-63.80377,13.956,0.025,13.09,0.025,10.752,0.09,0.809,0.019,4.323e-13,5.627e-14,2.338,0.093,0.866,0.035,0.03640386605255074,1.9360009700622371,,,,,,,,,,N,N,N,N,1.787804144301585e-07
J000450.84-691150.7,1eRASS J000450.0-691149,1.21186,-69.19743,13.887,0.025,13.556,0.029,11.949,0.243,0.862,0.02,8.156e-14,2.814e-14,1.607,0.245,0.331,0.038,0.023107286297843862,2.1547052988734734,,,,,,,,,,N,N,N,N,3.372965672200724e-08
J000504.33-414829.8,1eRASS J000504.6-414832,1.26808,-41.8083,14.048,0.026,13.684,0.033,12.14,0.378,0.743,0.018,7.22e-14,2.809e-14,1.544,0.379,0.364,0.042,0.08886641648213847,2.1513373440170045,,,,,,,,,,N,N,N,N,2.98587691923605e-08


In [250]:
## rounding 
wise_erosita['ra'] = [np.round(wise_erosita['ra'][i], 5) for i in range(len(wise_erosita))]
wise_erosita['dec'] = [np.round(wise_erosita['dec'][i], 5) for i in range(len(wise_erosita))]
wise_erosita['w1mpro'] = [np.round(wise_erosita['w1mpro'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['w1sigmpro'] = [np.round(wise_erosita['w1sigmpro'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['w2mpro'] = [np.round(wise_erosita['w2mpro'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['w2sigmpro'] = [np.round(wise_erosita['w2sigmpro'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['w3mpro'] = [np.round(wise_erosita['w3mpro'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['w3sigmpro'] = [np.round(wise_erosita['w3sigmpro'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['w1flux'] = wise_erosita['w1flux']*1E3
wise_erosita['w1flux'] = [np.round(wise_erosita['w1flux'][i], 3) for i in range(len(wise_erosita))] 
wise_erosita['w1sigflux'] = wise_erosita['w1sigflux']*1E3
wise_erosita['w1sigflux'] = [np.round(wise_erosita['w1sigflux'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['ML_FLUX_1'] = [float(f"{num:.3e}") for num in wise_erosita['ML_FLUX_1']]
wise_erosita['ML_FLUX_ERR_1'] = [float(f"{num:.3e}") for num in wise_erosita['ML_FLUX_ERR_1']]
wise_erosita['x'] = [np.round(wise_erosita['x'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['x_err'] = [np.round(wise_erosita['x_err'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['y'] = [np.round(wise_erosita['y'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['y_err'] = [np.round(wise_erosita['y_err'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['dist'] = [np.round(wise_erosita['dist'][i], 3) for i in range(len(wise_erosita))]
wise_erosita['Alpha_IRX'] = [np.round(wise_erosita['Alpha_IRX'][i], 3) for i in range(len(wise_erosita))]



In [252]:
wise_erosita.rename_column('x', 'w2mpro-w3mpro')
wise_erosita.rename_column('x_err', 'w2mpro-w3mpro_ERR')
wise_erosita.rename_column('y', 'w1mpro-w2mpro')
wise_erosita.rename_column('y_err', 'w1mpro-w2mpro_ERR')


In [257]:
wise_erosita.rename_column('name', 'WISE Name')
wise_erosita.rename_column('IAUNAME', 'eROSITA Name')

In [258]:
wise_erosita.sort('ra')

In [259]:
tbl= QTable(wise_erosita)
tbl['ra'] = tbl['ra']*u.degree 
tbl['dec'] = tbl['dec']*u.degree
tbl['w1mpro'] = tbl['w1mpro']*u.mag
tbl['w1sigmpro'] = tbl['w1sigmpro']*u.mag
tbl['w2mpro'] = tbl['w2mpro']*u.mag
tbl['w2sigmpro'] = tbl['w2sigmpro']*u.mag
tbl['w3mpro'] = tbl['w3mpro']*u.mag
tbl['w3sigmpro'] = tbl['w3sigmpro']*u.mag
tbl['w1flux'] = tbl['w1flux']*u.mJy
tbl['w1sigflux'] = tbl['w1sigflux']*u.mJy
tbl['ML_FLUX_1'] = tbl['ML_FLUX_1']*u.erg/(u.cm**2*u.s)
tbl['ML_FLUX_ERR_1'] = tbl['ML_FLUX_ERR_1']*u.erg/(u.cm**2*u.s)
tbl['w2mpro-w3mpro'] = tbl['w2mpro-w3mpro']*u.mag
tbl['w2mpro-w3mpro_ERR'] = tbl['w2mpro-w3mpro_ERR']*u.mag
tbl['w1mpro-w2mpro'] = tbl['w1mpro-w2mpro']*u.mag
tbl['w1mpro-w2mpro_ERR'] = tbl['w1mpro-w2mpro_ERR']*u.mag
table_hdu = fits.BinTableHDU(tbl) 
header = table_hdu.header
header['EXTNAME'] = 'ALL_eROSITA_WISE_MATCHES'
header['AUTHOR'] = 'Cassie Metzger' 
header['DATE'] = '2025-04-16' 
header['WAVELENG'] = 'Multi'
header['DATASRC'] = 'eROSITA, WISE'
header['DOI'] = '10.48550/arXiv.2501.12520' 
header['WAVELENG'] = 'Multi'
header['SELPRO'] = 'All sources in the eROSITA catalog with WISE counterparts within 5'' that have IR magnitudes less than 14.3, 13.8, 12.2 in the W1, W2, W3 bands respectively and IR colors than fall between 1.25-2.4 and 0.05-0.9 in the W2-W3 and W1-W2 bands,respectively.'
tbl.write('./catalogs/WISE_EROSITA_ALLmatches.fits', format = 'fits', overwrite = True)
print('Catalog saved as WISE_EROSITA_ALLmatches.fits')

Catalog saved as WISE_EROSITA_ALLmatches.fits


In [256]:
df = wise_erosita.to_pandas()
df.to_excel('./catalogs/WISE_EROSITA_ALLmatches.xlsx', index = False)
print('Catalog saved as WISE_EROSITA_ALLmatches.xlsx')

Catalog saved as WISE_EROSITA_ALLmatches.xlsx
